In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig

# BitsAndBytesConfig to quantize the model int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

llm_name = "mistralai/Mistral-7B-Instruct-v0.1"
model = AutoModelForCausalLM.from_pretrained(llm_name, device_map='auto', quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(llm_name)
tokenizer.pad_token = tokenizer.eos_token

/u/kirushi/.conda/envs/guardrails1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/u/kirushi/.conda/envs/guardrails1/lib/python3.9/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-02-15 00:07:57.133518: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-15 00:08:01.695460: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the foll

In [3]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
   

In [4]:
# Testing the loaded model
prompt = "[INST] Give me list atleast five popular leaders name [/INST]"

encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
model_inputs = encoded_input.to('cuda')

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

decoded_output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
response = decoded_output[0][len(prompt):]

decoded_output[0],response

('[INST] Give me list atleast five popular leaders name [/INST] 1. Elon Musk\n2. Angela Merkel\n3. Barack Obama\n4. Xi Jinping\n5. Oprah Winfrey',
 ' 1. Elon Musk\n2. Angela Merkel\n3. Barack Obama\n4. Xi Jinping\n5. Oprah Winfrey')

In [5]:
from datasets import load_dataset

# Load the dataset with train, test, and validation splits
dataset = load_dataset('Krooz/Campus_Recruitment_CSV')
dataset

Extracting data files: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1652.82it/s]
Generating train split: 7225 examples [00:00, 55377.21 examples/s]
Generating validation split: 1275 examples [00:00, 187580.68 examples/s]
Generating test split: 1500 examples [00:00, 190864.18 examples/s]


DatasetDict({
    train: Dataset({
        features: ['CGPA', 'Internships', 'Projects', 'Workshops/Certifications', 'AptitudeTestScore', 'SoftSkillsRating', 'ExtracurricularActivities', 'PlacementTraining', 'SSC_Marks', 'HSC_Marks', 'PlacementStatus'],
        num_rows: 7225
    })
    validation: Dataset({
        features: ['CGPA', 'Internships', 'Projects', 'Workshops/Certifications', 'AptitudeTestScore', 'SoftSkillsRating', 'ExtracurricularActivities', 'PlacementTraining', 'SSC_Marks', 'HSC_Marks', 'PlacementStatus'],
        num_rows: 1275
    })
    test: Dataset({
        features: ['CGPA', 'Internships', 'Projects', 'Workshops/Certifications', 'AptitudeTestScore', 'SoftSkillsRating', 'ExtracurricularActivities', 'PlacementTraining', 'SSC_Marks', 'HSC_Marks', 'PlacementStatus'],
        num_rows: 1500
    })
})

In [6]:
dataset['train'][0]

{'CGPA': 7.7,
 'Internships': 1,
 'Projects': 1,
 'Workshops/Certifications': 0,
 'AptitudeTestScore': 69,
 'SoftSkillsRating': 4.0,
 'ExtracurricularActivities': 'No',
 'PlacementTraining': 'No',
 'SSC_Marks': 55,
 'HSC_Marks': 69,
 'PlacementStatus': 'NotPlaced'}

In [6]:
import json

def data_to_text_prompt(x):
    
    x = dict(x)
    label = x['PlacementStatus']
    x.pop('PlacementStatus')
    json_data = json.dumps(x, indent=4)
    
    prompt = f"""[INST] Instruction:
Write an objective overview about the following colleage student based only on the provided structured data in the JSON format.
You should include details and cover the information mentioned in the placement data. The overview should be 100 - 200 words. 
Don’t make up information. Don't give any additional feedback just represent the given information in the overview. 
Use a random human name for the student, Dont start with 'based on the structured data'

Structured data:
{json_data}

- SSC_Marks denote marks attained by the student in senion secondary school
- HSC_Marks denote marks attained by the student in higher seconday school
- CGPA is the cummulative GPA attained by the student in his university
 
Overview:
[/INST]"""
    
    return {'prompt': prompt, 'label': label}

In [7]:
prompt = data_to_text_prompt(dataset['train'][0])['prompt']

encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
model_inputs = encoded_input.to('cuda')

generated_ids = model.generate(**model_inputs, max_new_tokens=500, do_sample=True, pad_token_id=tokenizer.eos_token_id)

decoded_output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(decoded_output[0][len(prompt):])

 Meet Maria. Maria is a current university student with a 7.7 CGPA, indicating strong academic performance in her college years. She has completed one internship and one project, demonstrating a practical application of her academic knowledge and a willingness to learn through real-world experiences. Maria does not have any work


In [8]:
new_datasets = dataset.map(
    data_to_text_prompt, batched=False, remove_columns=dataset["train"].column_names
)
new_datasets

DatasetDict({
    train: Dataset({
        features: ['prompt', 'label'],
        num_rows: 7225
    })
    test: Dataset({
        features: ['prompt', 'label'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['prompt', 'label'],
        num_rows: 1275
    })
})

In [9]:
new_datasets['train'][0]

{'prompt': '[INST] Instruction:\nWrite an objective overview about the following colleage student based only on the provided structured data in the JSON format.\nYou should include details and cover the information mentioned in the placement data. The overview should be 100 - 200 words. \nDon’t make up information. Don\'t give any additional feedback just represent the given information in the overview. \nUse a random human name for the student, Dont start with \'based on the structured data\'\n\nStructured data:\n{\n    "CGPA": 7.7,\n    "Internships": 1,\n    "Projects": 1,\n    "Workshops/Certifications": 0,\n    "AptitudeTestScore": 69,\n    "SoftSkillsRating": 4.0,\n    "ExtracurricularActivities": "No",\n    "PlacementTraining": "No",\n    "SSC_Marks": 55,\n    "HSC_Marks": 69\n}\n\n- SSC_Marks denote marks attained by the student in senion secondary school\n- HSC_Marks denote marks attained by the student in higher seconday school\n- CGPA is the cummulative GPA attained by the s

In [11]:
def overview_response(x):
    encoded_input = tokenizer(x['prompt'],  return_tensors="pt", add_special_tokens=True)
    model_inputs = encoded_input.to('cuda')
    
    generated_ids = model.generate(**model_inputs, max_new_tokens=500, do_sample=True, pad_token_id=tokenizer.eos_token_id)

    decoded_output = tokenizer.batch_decode(generated_ids[:, model_inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    
    return {'response': decoded_output}

In [ ]:
new_datasets = new_datasets.map(
    overview_response, batched=True, batch_size = 256
)
new_datasets

In [ ]:
new_datasets['train'].to_csv('text_dataset/train.csv')
new_datasets['test'].to_csv('text_dataset/test.csv')
new_datasets['validation'].to_csv('text_dataset/val.csv')